In [1]:
%load_ext autoreload
%autoreload 2

In [43]:
import sys 
sys.path.append("/n/home12/binxuwang/Github/DiffusionReasoning")
from tensorboard_utils import extract_all_runs, extract_tensorboard_data_from_run, extract_last_step_summary

In [3]:
import os
import numpy as np
import pandas as pd
import torch
from os.path import join
import pickle as pkl
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.autonotebook import trange, tqdm

/tmp/ipykernel_3737493/464873755.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange, tqdm


In [44]:
from rule_new_utils import infer_rule_from_sample_batch, compute_rule_statistics, infer_rule_statistics_from_sample_batch

### Paths

In [4]:
tabdir = "/n/home12/binxuwang/Github/DiffusionReasoning/Tables"
# figdir = "/n/home12/binxuwang/Github/DiffusionReasoning/Figures_newrule"

GPT_exproot = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/GPT2_raven"
DiT_exproot = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/DiT/results"
SiT_exproot = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/SiT/results"
SSM_exproot = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/Mamba_raven"
EDM_exproot = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/mini_edm/exps"

In [5]:
figroot = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Figures/DiffusionReasoning"
GPTfigdir = join(figroot, "GPT2_raven")
EDMfigdir = join(figroot, "EDM_raven")
DiTfigdir = join(figroot, "DiT_raven")
SSMfigdir = join(figroot, "SSM_raven")
SiTfigdir = join(figroot, "SiT_raven")

In [6]:
import pandas as pd
# Set the maximum column width to 100 characters
pd.set_option('display.max_colwidth', 100)

In [7]:
!mkdir  /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Figures/DiffusionReasoning/GPT2_raven
!mkdir  /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Figures/DiffusionReasoning/EDM_raven
!mkdir  /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Figures/DiffusionReasoning/DiT_raven
!mkdir  /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Figures/DiffusionReasoning/SSM_raven
!mkdir  {SiTfigdir}


mkdir: cannot create directory ‘/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Figures/DiffusionReasoning/GPT2_raven’: File exists
mkdir: cannot create directory ‘/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Figures/DiffusionReasoning/EDM_raven’: File exists
mkdir: cannot create directory ‘/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Figures/DiffusionReasoning/DiT_raven’: File exists
mkdir: cannot create directory ‘/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Figures/DiffusionReasoning/SSM_raven’: File exists
mkdir: cannot create directory ‘/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Figures/DiffusionReasoning/SiT_raven’: File exists


In [8]:
figdir = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Figures/DiffusionReasoning/Figure_memorization_inspection"

### Util functions for Rule learning dynamics 

Function to parse the experiment string

In [9]:
import re
def extract_and_convert(string):
    """Simple function to extract pattern like stream0_16 or stream16M and convert to float 0.16 or integer 16"""
    # Pattern to match the "0_16" part and convert to float 0.16
    match1 = re.search(r'stream(\d+)_(\d+)', string)
    if match1:
        num1 = str(match1.group(1))
        num2 = str(match1.group(2))
        result = float(f"{num1}.{num2}")
        return result
    
    # Pattern to match the "16M" part and convert to integer 16
    match2 = re.search(r'stream(\d+)M', string)
    if match2:
        result = int(match2.group(1))
        return result
    
    return None

Functions to load the samples and eval stats, plot the data

In [31]:
def sweep_collect_sample(expname, exproot, prefix=None):
    assert os.path.exists(join(exproot, expname, "samples")), expname  
    # print(sorted(os.listdir(join(exproot, expname, "samples"))))
    print(f"Extracted data from {join(exproot, expname, 'samples')}")
    print(f"example file:", os.listdir(join(exproot, expname, 'samples'))[0:10])
    # raise NotImplementedError("This function is not implemented yet")
    # for files with names like 'sample_rule_eval_995000.pt' find the one with largest number
    epoch_nums = sorted([int(f.split(prefix)[-1].split(".pt")[0]) for f in os.listdir(join(exproot, expname,'samples')) if not (prefix in f)])
    eval_col = {}
    for epoch_num in tqdm(epoch_nums):
        samples_eval = torch.load(join(exproot, expname, 'samples', f"{epoch_num:07d}.pt"))
        eval_col[epoch_num] = samples_eval
    return eval_col

In [10]:
def sweep_collect_eval_data(expname, exproot, prefix=None):
    assert os.path.exists(join(exproot, expname, "samples")), expname  
    # print(sorted(os.listdir(join(exproot, expname, "samples"))))
    print(f"Extracted data from {join(exproot, expname, 'samples')}")
    print(f"example file:", os.listdir(join(exproot, expname, 'samples'))[0])
    # for files with names like 'sample_rule_eval_995000.pt' find the one with largest number
    epoch_nums = sorted([int(f.split(prefix)[-1].split(".pt")[0]) for f in os.listdir(join(exproot, expname,'samples')) if prefix in f])
    eval_col = {}
    for epoch_num in tqdm(epoch_nums):
        samples_eval = torch.load(join(exproot, expname, 'samples', f"{prefix}{epoch_num}.pt"))
        eval_col[epoch_num] = samples_eval
    return eval_col

In [11]:
def extract_rule_list_from_eval_col(eval_col, is_abinit = False):
    epoch_list = sorted(list(eval_col.keys()))
    rule_list_all = []
    consistency_all = []
    for epoch in eval_col.keys():
        if is_abinit:
            rule_list_all.append(eval_col[epoch]['rule_col_list_abinit'])
            consistency_all.append((eval_col[epoch]['C3_list_abinit'], eval_col[epoch]['C2_list_abinit']))
        else:
            rule_list_all.append(eval_col[epoch]['rule_col_list'])
            consistency_all.append((eval_col[epoch]['C3_list'], eval_col[epoch]['C2_list']))
    rule_list_all = np.array(rule_list_all, dtype=object)
    consistency_all = np.array(consistency_all, dtype=object)
    print(rule_list_all.shape, consistency_all.shape)
    return epoch_list, rule_list_all, consistency_all


def extract_rule_list_from_eval_col_Diffusion(eval_col, ):
    print("diffusion model, just fetch ab init generation")
    epoch_list = sorted(list(eval_col.keys()))
    rule_list_all = []
    consistency_all = []
    for epoch in eval_col.keys():
        rule_list_all.append(eval_col[epoch]['rule_col'])
        consistency_all.append((eval_col[epoch]['c3_list'], eval_col[epoch]['c2_list']))

    rule_list_all = np.array(rule_list_all, dtype=object)
    consistency_all = np.array(consistency_all, dtype=object)
    print(rule_list_all.shape, consistency_all.shape)
    return epoch_list, rule_list_all, consistency_all

In [12]:
def format_rule_list_to_mat(rule_list_all, consistency_all):
    """Format the collected rule_list_all and consistency_all into rule_cnt_mat, cons3_rule_cnt_mat, cons2_rule_cnt_mat.
    for plotting purposes.

    Args:
        rule_list_all: np.array, dtype object, shape=(num_epoch, num_sample, 3). 
        consistency_all: np.array, dtype object, shape=(num_epoch, 2, num_sample). 
                    where the 2nd dimension is the consistency of C3 and C2 rules corrspondingly. 

    Returns:
        rule_cnt_mat: np.array, shape=(num_epoch, 40). 
        cons3_rule_cnt_mat: np.array, shape=(num_epoch, 40). 
        cons2_rule_cnt_mat: np.array, shape=(num_epoch, 40).
    """
    epoch_num = rule_list_all.shape[0]
    rule_pool_all = []
    for i in range(epoch_num): # trange
        rule_pool = np.concatenate(list(rule_list_all[i,:,:].flatten())).astype(int)
        rule_pool_all.append(rule_pool)
    # plot the number of rules == rule_i for each generation
    rule_cnt_mat = np.zeros((epoch_num, 40))
    for i in range(epoch_num): # trange
        rule_pool = rule_pool_all[i]
        rule_uniq, counts = np.unique(rule_pool, return_counts=True)
        rule_cnt_mat[i, rule_uniq] = counts

    cons3_rule_pool_all = []
    cons2_rule_pool_all = []
    for i in range(epoch_num): # trange
        rule_pool = np.concatenate(list(consistency_all[i,0,:].flatten())).astype(int)
        cons3_rule_pool_all.append(rule_pool)
        rule_pool = np.concatenate(list(consistency_all[i,1,:].flatten())).astype(int)
        cons2_rule_pool_all.append(rule_pool)

    cons3_rule_cnt_mat = np.zeros((epoch_num, 40))
    cons2_rule_cnt_mat = np.zeros((epoch_num, 40))
    for i in range(epoch_num): # trange
        rule_pool = cons3_rule_pool_all[i]
        rule_uniq, counts = np.unique(rule_pool, return_counts=True)
        cons3_rule_cnt_mat[i, rule_uniq] = counts
        rule_pool = cons2_rule_pool_all[i]
        rule_uniq, counts = np.unique(rule_pool, return_counts=True)
        cons2_rule_cnt_mat[i, rule_uniq] = counts
    return rule_cnt_mat, cons3_rule_cnt_mat, cons2_rule_cnt_mat

In [14]:
from rule_new_utils import rule_table, relation_dict, attribute_dict
from circuit_toolkit.plot_utils import saveallforms
def visualize_indiv_rule_dynam(epoch_list, rule_mat, conv_wid=10, heldout_id=[1, 16, 20, 34, 37],
                               titlestr="Valid rule count separated by rule type", ylabel="Count", axs=None):
    # remove top and right spines from plot with plt
    plt.rcParams.update({'font.size': 12})
    if axs is None:
        figh, axs = plt.subplots(4, 10, figsize=(30, 12.5), sharex=True, sharey=True)
    else:
        figh = axs[0,0].get_figure()
    axs_f = axs.flatten()
    for i in range(40):
        ax = axs_f[i]
        # smooth the curve
        smooth_rule_cnt = np.convolve(rule_mat[:,i], np.ones(conv_wid)/conv_wid, mode='same')
        ax.plot(epoch_list, smooth_rule_cnt, alpha=0.7, )
        ax.set_title(f"R{i}: {rule_table[i]}")
        # change the font color of title to red
        if i in heldout_id:
            ax.title.set_color('red')
        if i >= 30:
            ax.set_xlabel("generation")
        if i % 10 == 0:
            ax.set_ylabel(ylabel)
    figh.suptitle(titlestr, fontsize=20)
    figh.tight_layout()
    figh.show()
    return figh, axs

In [15]:
def visualize_train_heldout_rule(epoch_list, plot_mat, heldout_id, 
                                 normalizer=None, titlestr="", 
                                 reflevel=None):#conv_wid=10, axs=None):
    heldout_mask = np.zeros((40,)).astype(bool)
    heldout_mask[heldout_id] = True
    if normalizer is not None:
        plot_mat = plot_mat / normalizer
    figh, axs = plt.subplots(1, 2, figsize=(11, 5), sharey=True)
    plt.subplot(1, 2, 1)
    plt.plot(epoch_list, plot_mat[:, ~heldout_mask], alpha=0.4)
    plt.plot(epoch_list, plot_mat[:, ~heldout_mask].mean(axis=1), color='black', linewidth=2)
    if reflevel is not None:
        plt.axhline(y=reflevel, color='r', linestyle='--')
    plt.subplot(1, 2, 2)
    plt.plot(epoch_list, plot_mat[:, heldout_mask], alpha=0.4)
    plt.plot(epoch_list, plot_mat[:, heldout_mask].mean(axis=1), color='black', linewidth=2)
    if reflevel is not None:
        plt.axhline(y=reflevel, color='r', linestyle='--')
    plt.suptitle(titlestr) # "Accuracy of C3 for each rule type"
    plt.show()
    return figh

## SiT

In [13]:
syndf_SiT = pd.read_csv(join(tabdir, "SiT_raven_tensorboard_data.csv"), index_col=0)
tb_data_col_SiT = pkl.load(open(join(tabdir, "SiT_raven_tensorboard_raw_data.pkl"), "rb"))

In [18]:
syndf_SiT.full_name

run_name
004-SiT_S_1-stream0_16M_pilot-Linear-velocity-None            004-SiT_S_1-stream0_16M_pilot-Linear-velocity-None/tensorboard_logs
005-SiT_B_1-stream0_16M_pilot-Linear-velocity-None            005-SiT_B_1-stream0_16M_pilot-Linear-velocity-None/tensorboard_logs
006-SiT_S_1-stream0_016M_all-Linear-velocity-None              006-SiT_S_1-stream0_016M_all-Linear-velocity-None/tensorboard_logs
007-SiT_S_1-stream0_16M_all-Linear-velocity-None                007-SiT_S_1-stream0_16M_all-Linear-velocity-None/tensorboard_logs
008-SiT_S_1-stream1_6M_all-Linear-velocity-None                  008-SiT_S_1-stream1_6M_all-Linear-velocity-None/tensorboard_logs
010-SiT_S_1-stream16M_all-Linear-velocity-None                    010-SiT_S_1-stream16M_all-Linear-velocity-None/tensorboard_logs
011-SiT_B_1-stream0_16M_all-Linear-velocity-None                011-SiT_B_1-stream0_16M_all-Linear-velocity-None/tensorboard_logs
011-SiT_S_1-stream16M_heldout0-Linear-velocity-None          011-SiT_S_1-stream16

In [50]:
for expfullname in ["014-SiT_S_1-stream0_16M_heldout0-Linear-velocity-None/tensorboard_logs",]:
                    # "017-SiT_B_1-stream0_16M_heldout0-Linear-velocity-None/tensorboard_logs"
    tb_record = tb_data_col_SiT[expfullname]
    expname = expfullname.split("/tensorboard_logs")[0]
    prefix = "sample_rule_eval_" #"eval_step" if "stream" in expname else "eval_epoch"
    # eval_col = sweep_collect_eval_data(expname, SiT_exproot, prefix=prefix)
    sample_col = sweep_collect_sample(expname, SiT_exproot, prefix=prefix)

Extracted data from /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/SiT/results/014-SiT_S_1-stream0_16M_heldout0-Linear-velocity-None/samples
example file: ['0155000.pt', '0422500.pt', '0432500.pt', '0715000.pt', '0787500.pt', '0072500.pt', '0277500.pt', 'sample_rule_eval_465000.pt', 'sample_rule_eval_602500.pt', '0707500.pt']


  0%|          | 0/402 [00:00<?, ?it/s]

In [51]:
max(sample_col.keys())

1000000

In [55]:
C3_cnt, C2_cnt, valid_cnt, total_cnt = infer_rule_statistics_from_sample_batch(sample_col[1000000])
print("C3 fraction:", C3_cnt / total_cnt, "C2 fraction:", C2_cnt / total_cnt, "valid fraction:", valid_cnt / total_cnt / 3)

C3 fraction: 0.7119140625 C2 fraction: 0.12548828125 valid fraction: 0.8453776041666666


In [57]:
(valid_cnt/total_cnt/3)**2

0.7146632936265733

In [56]:
(valid_cnt/total_cnt/3)**3

0.6041603429518915

In [53]:
1458 / 2048

0.7119140625

In [58]:
!echo $STORE_DIR/Datasets/RPM_dataset/RPM1000k 

/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Datasets/RPM_dataset/RPM1000k


In [62]:
import einops
train_attr_fn = "attr_all_1000k.pt" 
data_root = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Datasets/RPM_dataset/RPM1000k"
def get_dataset(data_root, train_attr_fn, n_classes=40, cmb_per_class=4000, heldout_ids=()):
    train_attrs = torch.load(f'{data_root}/{train_attr_fn}') # [35, 10000, 3, 9, 3]
    attr_img_tsr = einops.rearrange(train_attrs,  'cls (B R) p (H W) attr -> cls B attr (R H) (p W)', H=3,W=3,p=3,R=3,attr=3,)
    max_default_cmb = attr_img_tsr.shape[1]
    if cmb_per_class > max_default_cmb:
        raise ValueError(f'cmb_per_class should be less than {max_default_cmb}')
    train_cls_msk = torch.ones(n_classes, dtype=bool)
    for heldout_id in heldout_ids:
        train_cls_msk[heldout_id] = False
    X = attr_img_tsr[:, :cmb_per_class]
    y = torch.arange(0, n_classes).unsqueeze(1).expand(n_classes, cmb_per_class).to(int)
    X = einops.rearrange(X[train_cls_msk, :], 'cls B attr H W -> (cls B) attr H W')
    y = einops.rearrange(y[train_cls_msk, :], 'cls B -> (cls B)')
    return X, y

In [63]:
train_tsr_X, train_tsr_y = get_dataset(data_root, train_attr_fn, n_classes=40, cmb_per_class=4000, heldout_ids=())

In [64]:
train_tsr_X.shape

torch.Size([160000, 3, 9, 9])

In [72]:
train_tsr_X_rows = einops.rearrange(train_tsr_X, 'B attr (R h) W -> (B R) (attr h W)', R=3, h=3, W=9).numpy().astype(int)
train_tsr_X_rows.shape

(480000, 81)

In [73]:
train_X_row_set = set([tuple(train_tsr_X_rows[i]) for i in range(train_tsr_X_rows.shape[0])])

In [79]:
gen_sample_rows = einops.rearrange(sample_col[1000000].round().int(), 'B attr (R h) W -> (B R) (attr h W)', R=3, h=3, W=9).numpy().astype(int)
gen_sample_rows_list = [tuple(gen_sample_rows[i]) for i in range(gen_sample_rows.shape[0])]

In [81]:
def check_fraction_train_set(gen_sample_rows_list, train_X_row_set):
    cnt = 0
    for i, row in enumerate(gen_sample_rows_list):
        if row in train_X_row_set:
            cnt += 1
            # print(i, row)
    return cnt, cnt / len(gen_sample_rows_list)


In [82]:
for epoch in tqdm(sample_col.keys()):
    gen_sample_rows = einops.rearrange(sample_col[epoch].round().int(), 'B attr (R h) W -> (B R) (attr h W)', R=3, h=3, W=9).numpy().astype(int)
    gen_sample_rows_list = [tuple(gen_sample_rows[i]) for i in range(gen_sample_rows.shape[0])]
    cnt, frac = check_fraction_train_set(gen_sample_rows_list, train_X_row_set)
    print(epoch, cnt, frac)

  0%|          | 0/402 [00:00<?, ?it/s]

0 0 0.0
1 0 0.0
2500 0 0.0
5000 0 0.0
7500 0 0.0
10000 0 0.0
12500 0 0.0
15000 0 0.0
17500 0 0.0
20000 0 0.0
22500 0 0.0
25000 0 0.0
27500 0 0.0
30000 0 0.0
32500 0 0.0
35000 0 0.0
37500 0 0.0
40000 0 0.0
42500 0 0.0
45000 0 0.0
47500 0 0.0
50000 0 0.0
52500 0 0.0
55000 0 0.0
57500 0 0.0
60000 0 0.0
62500 0 0.0
65000 0 0.0
67500 0 0.0
70000 0 0.0
72500 0 0.0
75000 0 0.0
77500 0 0.0
80000 0 0.0
82500 0 0.0
85000 0 0.0
87500 0 0.0
90000 0 0.0
92500 0 0.0
95000 0 0.0
97500 0 0.0
100000 0 0.0
102500 0 0.0
105000 0 0.0
107500 0 0.0
110000 0 0.0
112500 0 0.0
115000 0 0.0
117500 0 0.0
120000 0 0.0
122500 0 0.0
125000 0 0.0
127500 0 0.0
130000 0 0.0
132500 0 0.0
135000 0 0.0
137500 0 0.0
140000 0 0.0
142500 0 0.0
145000 0 0.0
147500 0 0.0
150000 0 0.0
152500 0 0.0
155000 0 0.0
157500 0 0.0
160000 0 0.0
162500 0 0.0
165000 0 0.0
167500 0 0.0
170000 0 0.0
172500 0 0.0
175000 0 0.0
177500 0 0.0
180000 0 0.0
182500 0 0.0
185000 0 0.0
187500 0 0.0
190000 0 0.0
192500 0 0.0
195000 0 0.0
197500 0 0.0

In [19]:
# for expfullname in syndf_SiT.full_name:
for expfullname in ["017-SiT_B_1-stream0_16M_heldout0-Linear-velocity-None/tensorboard_logs"]:
    tb_record = tb_data_col_SiT[expfullname]
    expname = expfullname.split("/tensorboard_logs")[0]
    prefix = "sample_rule_eval_" #"eval_step" if "stream" in expname else "eval_epoch"
    eval_col = sweep_collect_eval_data(expname, SiT_exproot, prefix=prefix)
    epoch_list, rule_list_all, consistency_all = extract_rule_list_from_eval_col_Diffusion(eval_col, )
    rule_cnt_mat, cons3_rule_cnt_mat, cons2_rule_cnt_mat = format_rule_list_to_mat(rule_list_all, consistency_all)

Extracted data from /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/SiT/results/017-SiT_B_1-stream0_16M_heldout0-Linear-velocity-None/samples
example file: 0155000.pt


  0%|          | 0/402 [00:00<?, ?it/s]

diffusion model, just fetch ab init generation
(402, 2048, 3) (402, 2, 2048)


In [21]:
type(eval_col)

dict

### plot all individual rule data

In [ ]:
for expfullname in syndf_SiT.full_name:
    tb_record = tb_data_col_SiT[expfullname]
    expname = expfullname.split("/tensorboard_logs")[0]
    prefix = "sample_rule_eval_" #"eval_step" if "stream" in expname else "eval_epoch"
    eval_col = sweep_collect_eval_data(expname, SiT_exproot, prefix=prefix)
    epoch_list, rule_list_all, consistency_all = extract_rule_list_from_eval_col_Diffusion(eval_col, )
    rule_cnt_mat, cons3_rule_cnt_mat, cons2_rule_cnt_mat = format_rule_list_to_mat(rule_list_all, consistency_all)
    eval_sample_num = rule_list_all.shape[1]
    print(expname)
    if "heldout0" in expname:
        heldout_id = [1, 16, 20, 34, 37]  
    else:
        heldout_id = []

    figh, axs = visualize_indiv_rule_dynam(epoch_list, rule_cnt_mat, conv_wid=10, heldout_id=heldout_id,
                            titlestr=f"{expname}\nValid rule count separated by rule type")
    saveallforms(SiTfigdir, f"{expname}_indiv_rule_validity", figh)

    figh, axs = visualize_indiv_rule_dynam(epoch_list, cons3_rule_cnt_mat, conv_wid=10, heldout_id=heldout_id,
                                        titlestr=f"{expname}\nConsistency 3 (blue) and 2 (orange) rule count", )
    figh, axs = visualize_indiv_rule_dynam(epoch_list, cons2_rule_cnt_mat, conv_wid=10, heldout_id=heldout_id,
                                        titlestr=f"{expname}\nConsistency 3 (blue) and 2 (orange) rule count", axs=axs)
    saveallforms(SiTfigdir, f"{expname}_indiv_rule_consistency", figh)
    
    figh = visualize_train_heldout_rule(epoch_list, rule_cnt_mat, heldout_id, 
                                normalizer=eval_sample_num * 3, reflevel=1 / 40,
                                titlestr=f"{expname}\nValid rule fraction")
    saveallforms(EDMfigdir, f"{expname}_train_held_rule_validity", figh)

    figh = visualize_train_heldout_rule(epoch_list, cons3_rule_cnt_mat, heldout_id, 
                                normalizer=eval_sample_num, reflevel=1 / 40,
                                titlestr=f"{expname}\nC3 sample fraction")
    saveallforms(SiTfigdir, f"{expname}_train_held_rule_consistency", figh)
    # raise ValueError("stop here")